In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


class TasksParser:
    def __init__(self):
        self.urls = [
            "https://www.potehechas.ru/zadachi/zadachi.shtml",
            "https://www.potehechas.ru/zadachi/zadachi_2.shtml",
            "https://www.potehechas.ru/zadachi/zadachi_3.shtml",
            "https://www.potehechas.ru/zadachi/zadachi_4.shtml",
            "https://www.potehechas.ru/zadachi/zadachi_5.shtml",
            "https://www.potehechas.ru/zadachi/zadachi_6.shtml",
            "https://www.potehechas.ru/zadachi/zadachi_7.shtml",
            "https://www.potehechas.ru/zadachi/zadachi_8.shtml",
            "https://www.potehechas.ru/zadachi/zadachi_9.shtml",
        ]
        
        self.dataset = []
        
    def parse_tasks(self):
        for url in self.urls:
            response = requests.get(url=url)
            soup = BeautifulSoup(response.text)
            all_tasks = soup.find_all("table", class_="comments_generall_table")
            for item in all_tasks:
            # print(item)
            # print("-"*50)
                try:
                    task_text = item.find("td", class_="zadacha_text").text.strip()
                    task_text = task_text.replace("«« Ответ к задаче »»", "")
                    task_text = task_text.replace("\r", "")
                    print(task_text)
                    answer_text = (
                        item
                        .find("div", class_="otvet_zadachi_in")["onmousedown"]
                        .replace("this.innerHTML=", "")
                    )
                    answer_text = answer_text.replace("'", "")
                    answer_text = answer_text.replace("<br>", "\n")
                    print(answer_text)
                    
                    title = item.find("td", class_="comments_info_2").text.strip()
                    print(title)
                    print("=" * 50)
                    
                    self.dataset.append({
                        "title": title,
                        "answer": answer_text,
                        "task": task_text
                    })
                except:
                    # print(item)
                    pass
        
        pd.DataFrame(data=self.dataset).to_csv("./verbalist/datasets/logic_tasks_ru/logic_tasks_ru.csv", index=False)

In [ ]:
parser = TasksParser()

parser.parse_tasks()

### convert to datasets

In [4]:
import pandas as pd
from datasets import Dataset

path = "./verbalist/datasets/logic_tasks_ru/logic_tasks_ru - labeled.csv"
dataset = pd.read_csv(path)
dataset = dataset.to_dict("records")
dataset = Dataset.from_list(dataset)
# dataset.push_to_hub("dim/logic_tasks_ru")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


### prepare for training

In [3]:
from datasets import load_dataset

dataset = load_dataset("dim/logic_tasks_ru")
dataset = dataset['train']

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--logic_tasks_ru-9092f50679ed969e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 858.08it/s]


In [4]:
dataset[0]

{'title': 'Кувшинки на пруду',
 'task': 'На поверхности пруда плавает одна кувшинка, которая постоянно делится и разрастается. \nТаким образом, каждый день площадь, которую занимают кувшинки, увеличивается в два раза. Через месяц покрытой оказывается вся \nповерхность пруда. За сколько времени покроется кувшинками вся поверхность пруда, если изначально на поверхности будут плавать две кувшинки? \n\n',
 'answer': 'Две кувшинки покроют озеро за месяц минус один день.',
 'ok/trash': 'trash'}